In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Conv1D, MaxPooling1D, Flatten, LSTM, TimeDistributed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow.keras.optimizers import Adam


In [2]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
main_directory = os.path.dirname(parent_directory)
phase_1 =  pd.read_csv(main_directory + '/data/phase_1_data.csv')
phase_1.head()

,ID,From,Date,Hour,Participation_Phase,Demand_kWh,Temperature,Temperature24,Temperature48,Temperature72
0,Exp_553,2020-02-01 00:00:00+00:00,2020-02-01,1,Phase_1,1.174,7.6,6.1,5.0,4.4
1,Exp_553,2020-02-01 01:00:00+00:00,2020-02-01,2,Phase_1,1.172,8.2,6.2,5.1,4.4
2,Exp_553,2020-02-01 02:00:00+00:00,2020-02-01,3,Phase_1,1.207,8.4,6.3,5.3,4.5
3,Exp_553,2020-02-01 03:00:00+00:00,2020-02-01,4,Phase_1,1.613,8.4,6.4,5.4,4.5
4,Exp_553,2020-02-01 04:00:00+00:00,2020-02-01,5,Phase_1,1.101,8.2,6.5,5.5,4.6


In [3]:
# phase_1['From'] = pd.to_datetime(phase_1['From'])
# phase_1['Day'] = phase_1['From'].dt.day
# phase_1['Month'] = phase_1['From'].dt.month
# phase_1['Weekday'] = phase_1['From'].dt.weekday
phase_1.set_index('From',inplace=True)

phase_1.head()

,ID,Date,Hour,Participation_Phase,Demand_kWh,Temperature,Temperature24,Temperature48,Temperature72
From,,,,,,,,,
2020-02-01 00:00:00+00:00,Exp_553,2020-02-01,1,Phase_1,1.174,7.6,6.1,5.0,4.4
2020-02-01 01:00:00+00:00,Exp_553,2020-02-01,2,Phase_1,1.172,8.2,6.2,5.1,4.4
2020-02-01 02:00:00+00:00,Exp_553,2020-02-01,3,Phase_1,1.207,8.4,6.3,5.3,4.5
2020-02-01 03:00:00+00:00,Exp_553,2020-02-01,4,Phase_1,1.613,8.4,6.4,5.4,4.5
2020-02-01 04:00:00+00:00,Exp_553,2020-02-01,5,Phase_1,1.101,8.2,6.5,5.5,4.6


75/25

In [4]:
df_features = ['Temperature']
df_target = ["Demand_kWh"]

X= phase_1[df_features].values
y=phase_1[df_target].values

X = X.reshape((X.shape[0], 1, X.shape[1])) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

In [5]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[1])).reshape(X_train.shape)
X_test = scaler_X.transform(X_test.reshape(-1, X_test.shape[1])).reshape(X_test.shape)

y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

In [6]:
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(1, 1))) 
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='mse')

c:\Users\alfre\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
history = model.fit(X_train, y_train,epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
4557/4557 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 0.0154 - val_loss: 0.0170
Epoch 2/10
4557/4557 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.0153 - val_loss: 0.0172
Epoch 3/10
4557/4557 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0150 - val_loss: 0.0173
Epoch 4/10
4557/4557 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0151 - val_loss: 0.0167
Epoch 5/10
4557/4557 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0154 - val_loss: 0.0170
Epoch 6/10
4557/4557 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0154 - val_loss: 0.0172
Epoch 7/10
4557/4557 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0153 - val_loss: 0.0170
Epoch 8/10
4557/4557 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0151 - val_loss: 0.0175
Epoch 9/10
4557/4557 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0151 - val_loss: 0.0173
Epoch 10/10
4557/4557 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0152 - val_loss: 0.0172


In [8]:
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

forecast = model.predict(X_test)
y_pred = scaler_y.inverse_transform(forecast)

forecast_df = pd.DataFrame(data=y_pred, columns=['forecast_data'], index=phase_1.index[:len(y_pred)])

forecast_df = phase_1.join(forecast_df, how="left")

print(forecast_df)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

760/760 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step - loss: 0.0157
Test Loss: 0.017177226021885872
760/760 ━━━━━━━━━━━━━━━━━━━━ 1s 733us/step
                                ID        Date  Hour Participation_Phase  \
From                                                                       
2020-02-01 00:00:00+00:00  Exp_553  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00  Exp_553  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00  Exp_553  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00  Exp_553  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00  Exp_553  2020-02-01     1             Phase_1   
...                            ...         ...   ...                 ...   
2020-03-20 23:00:00+00:00  Exp_460  2020-03-20    24             Phase_1   
2020-03-20 23:00:00+00:00  Exp_460  2020-03-20    24             Phase_1   
2020-03-20 23:00:00+00:00  Exp_460  2020-03-20    24             Phase_1   
2020-03-20 23:00:00+00:00  Exp_

In [9]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Temperature24","Temperature48","Temperature72","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Neural_Network/predictions/phase_1/cnn_lstm_phase_1_method_2_75_25.csv",index=True)

80/20

In [10]:
df_features = ['Temperature']
df_target = ["Demand_kWh"]

X= phase_1[df_features].values
y=phase_1[df_target].values

X = X.reshape((X.shape[0], 1, X.shape[1])) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [11]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[1])).reshape(X_train.shape)
X_test = scaler_X.transform(X_test.reshape(-1, X_test.shape[1])).reshape(X_test.shape)

y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

In [12]:
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(1, 1))) 
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='mse')

c:\Users\alfre\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
history = model.fit(X_train, y_train,epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
4860/4860 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.0152 - val_loss: 0.0197
Epoch 2/10
4860/4860 ━━━━━━━━━━━━━━━━━━━━ 5s 954us/step - loss: 0.0152 - val_loss: 0.0194
Epoch 3/10
4860/4860 ━━━━━━━━━━━━━━━━━━━━ 5s 929us/step - loss: 0.0147 - val_loss: 0.0188
Epoch 4/10
4860/4860 ━━━━━━━━━━━━━━━━━━━━ 4s 905us/step - loss: 0.0146 - val_loss: 0.0191
Epoch 5/10
4860/4860 ━━━━━━━━━━━━━━━━━━━━ 4s 901us/step - loss: 0.0148 - val_loss: 0.0192
Epoch 6/10
4860/4860 ━━━━━━━━━━━━━━━━━━━━ 4s 906us/step - loss: 0.0150 - val_loss: 0.0185
Epoch 7/10
4860/4860 ━━━━━━━━━━━━━━━━━━━━ 4s 897us/step - loss: 0.0150 - val_loss: 0.0189
Epoch 8/10
4860/4860 ━━━━━━━━━━━━━━━━━━━━ 4s 900us/step - loss: 0.0151 - val_loss: 0.0188
Epoch 9/10
4860/4860 ━━━━━━━━━━━━━━━━━━━━ 4s 908us/step - loss: 0.0150 - val_loss: 0.0192
Epoch 10/10
4860/4860 ━━━━━━━━━━━━━━━━━━━━ 4s 899us/step - loss: 0.0148 - val_loss: 0.0188


In [14]:
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

forecast = model.predict(X_test)
y_pred = scaler_y.inverse_transform(forecast)

forecast_df = pd.DataFrame(data=y_pred, columns=['forecast_data'], index=phase_1.index[:len(y_pred)])

forecast_df = phase_1.join(forecast_df, how="left")

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

608/608 ━━━━━━━━━━━━━━━━━━━━ 1s 665us/step - loss: 0.0192
Test Loss: 0.018766017630696297
608/608 ━━━━━━━━━━━━━━━━━━━━ 1s 740us/step
MSE: 3.657292767344886
MAE: 1.9074842385664712
RMSE: 1.9124049694938794


In [15]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Temperature24","Temperature48","Temperature72","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Neural_Network/predictions/phase_1/cnn_lstm_phase_1_method_2_80_20.csv",index=True)

50/50

In [16]:
df_features = ['Temperature']
df_target = ["Demand_kWh"]

X= phase_1[df_features].values
y=phase_1[df_target].values

X = X.reshape((X.shape[0], 1, X.shape[1])) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)

In [17]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[1])).reshape(X_train.shape)
X_test = scaler_X.transform(X_test.reshape(-1, X_test.shape[1])).reshape(X_test.shape)

y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

In [18]:
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(1, 1))) 
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='mse')

c:\Users\alfre\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
history = model.fit(X_train, y_train,epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
3038/3038 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.0172 - val_loss: 0.0183
Epoch 2/10
3038/3038 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0160 - val_loss: 0.0167
Epoch 3/10
3038/3038 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0162 - val_loss: 0.0169
Epoch 4/10
3038/3038 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0161 - val_loss: 0.0180
Epoch 5/10
3038/3038 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0162 - val_loss: 0.0179
Epoch 6/10
3038/3038 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0160 - val_loss: 0.0190
Epoch 7/10
3038/3038 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0163 - val_loss: 0.0174
Epoch 8/10
3038/3038 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0162 - val_loss: 0.0176
Epoch 9/10
3038/3038 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0161 - val_loss: 0.0170
Epoch 10/10
3038/3038 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0162 - val_loss: 0.0195


In [20]:
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

forecast = model.predict(X_test)
y_pred = scaler_y.inverse_transform(forecast)

forecast_df = pd.DataFrame(data=y_pred, columns=['forecast_data'], index=phase_1.index[:len(y_pred)])

forecast_df = phase_1.join(forecast_df, how="left")

print(forecast_df)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

1519/1519 ━━━━━━━━━━━━━━━━━━━━ 1s 630us/step - loss: 0.0220
Test Loss: 0.019538866356015205
1519/1519 ━━━━━━━━━━━━━━━━━━━━ 1s 669us/step
                                ID        Date  Hour Participation_Phase  \
From                                                                       
2020-02-01 00:00:00+00:00  Exp_553  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00  Exp_553  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00  Exp_553  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00  Exp_553  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00  Exp_553  2020-02-01     1             Phase_1   
...                            ...         ...   ...                 ...   
2020-03-20 23:00:00+00:00  Exp_460  2020-03-20    24             Phase_1   
2020-03-20 23:00:00+00:00  Exp_460  2020-03-20    24             Phase_1   
2020-03-20 23:00:00+00:00  Exp_460  2020-03-20    24             Phase_1   
2020-03-20 23:00:00+00:00  

In [21]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Temperature24","Temperature48","Temperature72","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Neural_Network/predictions/phase_1/cnn_lstm_phase_1_method_2_50_50.csv",index=True)